## Combine GOES netCDFs into one .zarr file

This could also be done using the `goes-ortho.build_zarr()` function but that is currently not working.

Clinton Alden
27 March 2025

In [10]:
import xarray as xr
import os
import dask
from dask.distributed import Client

In [11]:
def goes_nc_to_zarr(in_dir, out_dir, out_name):    
# Directory containing the NetCDF files
    nc_dir = in_dir
    
    # Recursively list all NetCDF files in the directory and subdirectories
    nc_files = []
    for root, dirs, files in os.walk(nc_dir):
        for file in files:
            print(file)
            if file.endswith('.nc'):
                nc_files.append(os.path.join(root, file))

    # Open multiple NetCDF files as a list of datasets
    datasets = [xr.open_dataset(f) for f in nc_files]

    # Concatenate datasets along the 't' coordinate
    combined_ds = xr.concat(datasets, dim='t')

    # Save the combined dataset to a Zarr file
    out_name = out_name
    combined_ds.to_zarr(out_dir+out_name)

In [12]:
day_of_month = '1'
in_dir = f'/storage/cdalden/goes/goes16/2023/6/{day_of_month}/'
out_dir = '/storage/cdalden/goes/goes16/2023/'
out_name = f'goes16_C13_colorado_2023060{day_of_month}.zarr'

goes_nc_to_zarr(in_dir, out_dir, out_name)

OR_ABI-L1b-RadC-M6C13_G16_s20231520001174_e20231520003558_c20231520004038.nc
OR_ABI-L1b-RadC-M6C13_G16_s20231520006174_e20231520008561_c20231520009003.nc
OR_ABI-L1b-RadC-M6C13_G16_s20231520011174_e20231520013558_c20231520014015.nc
OR_ABI-L1b-RadC-M6C13_G16_s20231520016174_e20231520018559_c20231520019015.nc
OR_ABI-L1b-RadC-M6C13_G16_s20231520021174_e20231520023558_c20231520024030.nc
OR_ABI-L1b-RadC-M6C13_G16_s20231520026174_e20231520028558_c20231520029001.nc
OR_ABI-L1b-RadC-M6C13_G16_s20231520031174_e20231520033558_c20231520034019.nc
OR_ABI-L1b-RadC-M6C13_G16_s20231520036174_e20231520038558_c20231520039034.nc
OR_ABI-L1b-RadC-M6C13_G16_s20231520041174_e20231520043559_c20231520043599.nc
OR_ABI-L1b-RadC-M6C13_G16_s20231520046174_e20231520048558_c20231520049037.nc
OR_ABI-L1b-RadC-M6C13_G16_s20231520051174_e20231520053558_c20231520054012.nc
OR_ABI-L1b-RadC-M6C13_G16_s20231520056174_e20231520058558_c20231520059012.nc
OR_ABI-L1b-RadC-M6C13_G16_s20231520101174_e20231520103560_c20231520104041.nc

: 

In [7]:
def goes_nc_to_zarr(in_dir, out_dir):
    # List all day directories in the input directory
    day_dirs = [d for d in os.listdir(in_dir) if os.path.isdir(os.path.join(in_dir, d))]

    for day_dir in day_dirs:
        day_path = os.path.join(in_dir, day_dir)
        
        # Recursively list all NetCDF files in the day directory
        nc_files = []
        for root, dirs, files in os.walk(day_path):
            for file in files:
                if file.endswith('.nc'):
                    nc_files.append(os.path.join(root, file))

        # Filter out problematic files
        valid_nc_files = []
        for file in nc_files:
            try:
                xr.open_dataset(file)
                valid_nc_files.append(file)
            except Exception as e:
                print(f"Skipping file {file} due to error: {e}")

        if valid_nc_files:
            # Open multiple NetCDF files as a single Xarray dataset using Dask
            ds = xr.open_mfdataset(valid_nc_files, combine='nested', concat_dim='time', parallel=True, chunks={'time': 10})

            # Save the combined dataset to a Zarr file
            out_name = f'goes16_C13_colorado_202306{day_dir}.zarr'
            ds.to_zarr(os.path.join(out_dir, out_name), mode='w')

In [9]:
in_dir = '/storage/cdalden/goes/goes16/2023/6/'
out_dir = '/storage/cdalden/goes/goes16/2023/'
out_name = 'goes16_C13_colorado_20230601_20230620.zarr'

# Start a Dask client
client = Client(memory_limit='25GB')

# Now run the same code as above
goes_nc_to_zarr(in_dir, out_dir)

/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35504 instead
  warnings.warn(
*** Error in `/home/cdalden/miniforge3/envs/goesenv/bin/python': free(): invalid next size (fast): 0x00007f1b225f6020 ***
2025-03-29 12:37:40,482 - distributed.nanny - WARNING - Restarting worker
2025-03-29 12:37:40,506 - distributed.nanny - WARNING - Restarting worker
2025-03-29 12:37:40,520 - distributed.nanny - WARNING - Restarting worker
2025-03-29 12:37:40,623 - distributed.nanny - WARNING - Restarting worker
2025-03-29 12:37:40,656 - distributed.nanny - WARNING - Restarting worker
2025-03-29 12:37:40,843 - distributed.nanny - WARNING - Restarting worker
2025-03-29 12:37:44,062 - distributed.nanny - WARNING - Restarting worker
2025-03-29 12:37:44,066 - distributed.nanny - WARNING - Restarting worker
2025-03-29 12:37:44,103 - distributed.nanny -

KilledWorker: Attempted to run task 'open_dataset-c66e0e43-a212-4aa9-b440-ee6b9f33922e' on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:41899. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

2025-03-29 12:37:51,222 - distributed.scheduler - ERROR - Task open_dataset-c1eb2d03-3a08-4032-9b09-f4dd19c95595 marked as failed because 4 workers died while trying to run it
2025-03-29 12:37:51,223 - distributed.scheduler - ERROR - Task open_dataset-69be511f-368c-4d24-b7b3-11a4ff4f9e5e marked as failed because 4 workers died while trying to run it
2025-03-29 12:37:51,224 - distributed.scheduler - ERROR - Task open_dataset-6927c53d-9f7b-4142-80ff-69db6a993417 marked as failed because 4 workers died while trying to run it
2025-03-29 12:37:51,225 - distributed.scheduler - ERROR - Task open_dataset-b549d758-a621-4070-abc9-89c798b75a8c marked as failed because 4 workers died while trying to run it
2025-03-29 12:37:51,227 - distributed.scheduler - ERROR - Task open_dataset-1e969949-99d7-438d-9a76-4802ace29e19 marked as failed because 4 workers died while trying to run it
2025-03-29 12:37:51,229 - distributed.scheduler - ERROR - Task open_dataset-7f6919ca-fd06-4770-8948-a31b0c655da0 marked 

There are 1 HDF5 objects open!

Report: open objects on 72057594037927936


2025-03-29 12:37:51,387 - distributed.nanny - WARNING - Restarting worker


In [3]:
import os
import pwd

def get_swap_usage():
    swap_usage = {}
    for pid in os.listdir('/proc'):
        if pid.isdigit():
            try:
                with open(f'/proc/{pid}/status') as f:
                    lines = f.readlines()
                    name = ""
                    swap = 0
                    uid = ""
                    for line in lines:
                        if line.startswith('Name:'):
                            name = line.split()[1]
                        if line.startswith('VmSwap:'):
                            swap = int(line.split()[1])
                        if line.startswith('Uid:'):
                            uid = line.split()[1]
                    if swap > 0:
                        user = pwd.getpwuid(int(uid)).pw_name
                        swap_usage[pid] = (name, swap, user)
            except FileNotFoundError:
                continue
    return swap_usage

def print_swap_usage(swap_usage):
    print(f"{'PID':>6} {'COMMAND':<20} {'SWAP (kB)':>10} {'USER':<10}")
    for pid, (name, swap, user) in sorted(swap_usage.items(), key=lambda item: item[1][1], reverse=True):
        print(f"{pid:>6} {name:<20} {swap:>10} {user:<10}")

swap_usage = get_swap_usage()
print_swap_usage(swap_usage)

   PID COMMAND               SWAP (kB) USER      
 27832 python                  3603208 dlhogan   
  4055 python                  2316012 dlhogan   
  5082 python                  1616228 dlhogan   
  4558 python                  1405124 dlhogan   
 26698 python                   862840 dlhogan   
  7439 python                   657332 elilouis  
 14392 python                   651592 elilouis  
 16258 python                   606212 elilouis  
 16254 python                   602720 elilouis  
 16247 python                   601708 elilouis  
 16248 python                   600880 elilouis  
 16256 python                   592116 elilouis  
 16259 python                   590172 elilouis  
 16262 python                   582516 elilouis  
 16257 python                   555804 elilouis  
 16260 python                   553332 elilouis  
 16244 python                   547256 elilouis  
 16252 python                   533876 elilouis  
 16261 python                   530444 elilouis  
